In [1]:
import time
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from suds.client import Client

# Fonction pour interagir avec le service SOAP
def clientFunction(message):
    try:
        # Créez un client SOAP pour appeler le service LoanDemand
        client = Client("http://localhost:8004/LoanDemandService?wsdl")
        response = client.service.loanDemand(message)
        
        # Affiche et enregistre la réponse du service
        print("Client Response:", response)
        with open("log.txt", "a") as log_file:
            log_file.write(f"Response for message '{message}': {response}\n")
        
    except Exception as e:
        print("An error occurred:", str(e))

# Fonction pour traiter le fichier détecté par Watchdog
def process_file(file_path):
    print(f"Processing file: {file_path}")
    try:
        # Lire le contenu du fichier
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            print("File content:", content)
            
            # Appeler la fonction clientFunction avec le contenu du fichier
            clientFunction(content)
        
        # Déplacer le fichier traité vers un dossier d'archivage
        archive_folder = "data/processed/"
        os.makedirs(archive_folder, exist_ok=True)
        os.rename(file_path, os.path.join(archive_folder, os.path.basename(file_path)))
        print(f"Moved {file_path} to {archive_folder}")
        
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

# Gestionnaire d'événements pour surveiller les nouveaux fichiers
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Vérifiez que l'événement correspond à un fichier
        if not event.is_directory:
            process_file(event.src_path)

# Chemin du dossier à surveiller
folder_to_watch = "data/"

# Initialiser et démarrer l'observateur
observer = Observer()
event_handler = MyHandler()
observer.schedule(event_handler, folder_to_watch, recursive=False)
observer.start()

print(f"Watching folder: {os.path.abspath(folder_to_watch)}")

# Garder le script en cours d'exécution pour surveiller le dossier
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping observer...")
    observer.stop()
observer.join()
print("Observer stopped.")


Watching folder: c:\Users\21627\Desktop\TP\data
Processing file: data/sarraaq.txt
File content: Nom du Client: John Doe
Adresse: 123 Rue de la Liberté, 75001 Paris, France
Email: john.doe@email.com
Numéro de Téléphone: +33 123 456 789
Montant du Prêt Demandé: 200000 EUR
Durée du Prêt: 20 ans
Description de la Propriété: Maison à deux étages
avec jardin, située dans un quartier résidentiel calme.
Revenu Mensuel: 5000 EUR
Dépenses Mensuelles: 3000 EUR
Client Response: Erreur de conversion : Expecting value: line 1 column 1 (char 0)
Moved data/sarraaq.txt to data/processed/
Stopping observer...
Observer stopped.
